# Part 1

In [1]:
import random 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
from bs4 import BeautifulSoup
!pip install html5lib
!pip install lxml

print('Everything imported.')

Everything imported.


Scrape the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M .
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
df= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
toronto_NH = df[0]
toronto_NH.columns = ['PostalCode','Borough','Neighborhood']
toronto_NH.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


 Use the .shape method to print the number of rows of the original dataframe.

In [3]:
toronto_NH.shape

(180, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [4]:
toronto_table = toronto_NH[toronto_NH.Borough != 'Not assigned'].reset_index(drop=True)
toronto_table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a cell has a borough but a "Not assigned"  neighborhood, then the neighborhood will be the same as the borough. 
But this does not apply.

In [5]:
Not_assigned = toronto_table['Neighborhood'] == "Not assigned"
toronto_table.loc[Not_assigned, 'Neighborhood'] = toronto_table.loc[Not_assigned, 'Borough']
toronto_table[Not_assigned]

,PostalCode,Borough,Neighborhood


Here I used the .shape method to print the number of rows of my dataframe.

In [6]:
toronto_table.shape

(103, 3)

# Part 2

 Import the csv file that has the geographical coordinates of each postal code from https://cocl.us/Geospatial_data.
 I used the method .head() to have a look at the dataframe.

In [7]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_90e381095a1e4558b2df2cd2d4d19226 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eSMR8kAO7DX0VwSuCoykiPq7xq5nxk8TbLP-xsulX6Cn',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_90e381095a1e4558b2df2cd2d4d19226.get_object(Bucket='egmentingandclusteringneighborhoo-donotdelete-pr-ujs6qrolst7nwm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Geospatial_Coordinates = pd.read_csv(body)
Geospatial_Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I renamed the column "Postal Code" to "PostalCode" so that they are named identical.
Then I merged the 2 tables and used the method .head() to have a look at the dataframe.

In [8]:
Geospatial_Coordinates.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
toronto_table_merged = pd.merge(toronto_table, Geospatial_Coordinates, on='PostalCode', how='inner')
toronto_table_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3

In order to define an instance of the geocoder, I need to define a user_agent. I have named my agent 'totonto'.
Afterwards I get the coordinates of Toronto and display them.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Now I plot the date into the toronto_map. First I create map of Toronto using latitude and longitude values, thebn I add the markers to the map.

In [10]:
# Explore and cluster the neighborhoods in Toronto
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_table_merged['Latitude'], toronto_table_merged['Longitude'], toronto_table_merged['Borough'], toronto_table_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 
toronto_map

For illustration purposes, I simplify the above map and segment and cluster only the neighborhoods in North York. So I slice the original dataframe and create a new dataframe of the North York data.

In [11]:
NorthYork_data = toronto_table_merged[toronto_table_merged['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [12]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of North York are 43.7543263, -79.44911696639593.


In [13]:
# Explore and cluster North York, Toronto
NorthYork_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Borough'], NorthYork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NorthYork_map) 
NorthYork_map